# TP3 - Parte 3: Modelo 1 - XGBoost

Para XGBoost, me di cuenta que el embedding TF-IDF no estaba dando buenos resultados. Asique lo cambié por un BERTweet embedding, eso se realizó en este colab pero fue movido al collab de [FeatureEngineeringAvanzado](https://colab.research.google.com/drive/1Bw-J2DlBCxO0wMxY0KQ8wTtN_Yu-KIOk?usp=sharing) para mayor prolijidad.

## Imports y descarga de dependencias

In [ ]:
import pandas as pd
import numpy as np

In [1]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from scipy.stats import uniform, randint
from xgboost import XGBClassifier

In [2]:
from category_encoders import TargetEncoder

## Load de datos:

Cargo los X e Y:

In [ ]:
X_train = pd.read_csv("../data/processed/X_train_procesado.csv", index_col=0)
y_train = pd.read_csv("../data/processed/y_train_procesado.csv", index_col=0)

X_validation = pd.read_csv("../data/processed/X_validation_procesado.csv", index_col=0)
y_validation = pd.read_csv("../data/processed/y_validation_procesado.csv", index_col=0)

X_test = pd.read_csv("../data/processed/X_test_procesado.csv", index_col=0)

Y para XGBoost voy a utilizar el embedding que hicimos con BERTweet, como vimos antes:

In [ ]:
df_train_bert = pd.read_csv("../data/processed/df_train_bert.csv", index_col=0)
df_valid_bert = pd.read_csv("../data/processed/df_valid_bert.csv", index_col=0)

In [ ]:
X_train = pd.concat([X_train, df_train_bert], axis=1)
X_validation = pd.concat([X_validation, df_valid_bert], axis=1)

In [ ]:
df_test_bert = pd.read_csv("../data/processed/df_test_bert.csv", index_col=0)
X_test = pd.concat([X_test, df_test_bert], axis=1)

Dropeo los textos porque ya los embedee con BERTweet

In [ ]:
X_train = X_train.drop(columns=["text"])
X_validation = X_validation.drop(columns=["text"])

In [ ]:
X_test = X_test.drop(columns=["text"])

## ColumnTransformer para XGBoost

Catalogo las columnas para más facilidad a la hora de recordar cómo transformar cáda una:

In [ ]:
categorical_keyword = ["keyword"]
categorical_tweetlen = ["tweet_length"]
categorical_location = ["standard_location"]
numerical_cols = [
  "num_uppercase_letters",
  "num_uppercase_words",
  "num_special_chars",
  "num_digits",
  "num_hashtags",
  "num_urls",
  "num_tags",
  "prop_digits",
  "prop_words"
]

Para tweetlen, quería aplicar un Binary encoder para preservar distancias entre las categorias (porque claramente las hay), pero ColumnTransformer me chistaba haciendo Pipeline del encoder. Asique directamente hice un One-Hot que es más practico.

In [ ]:
def frequency_encode(X):
    s = pd.Series(X.iloc[:,0])
    freq = s.value_counts(normalize=True)
    return s.map(freq).to_frame()

freq_encoder = FunctionTransformer(frequency_encode, feature_names_out="one-to-one")

In [ ]:
preprocessor = ColumnTransformer([
    ("keyword_ohe", OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_keyword),
    ("tweetlen_ohe", OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_tweetlen),
    ("location_tme", TargetEncoder(handle_unknown='value', handle_missing='value'), categorical_location),
    ("location_fe", freq_encoder, categorical_location),
    ("numerical_scaler", StandardScaler(), numerical_cols),
], remainder="passthrough")

In [ ]:
xgb_model = XGBClassifier(eval_metric='logloss', random_state = 123)

In [ ]:
xgb_pipeline = Pipeline([
    ('prep', preprocessor),
    ('model', xgb_model)
])

Sets X transformados para grid search sin pipeline:

In [ ]:
X_train_transformed = preprocessor.fit_transform(X_train, y_train)
X_validation_transformed = preprocessor.transform(X_validation)

In [ ]:
X_test_transformed = preprocessor.transform(X_test)

## RandomizedSearch Inicial:

In [ ]:
xgb_random_params = {
    'model__n_estimators': randint(150, 400),
    'model__learning_rate': uniform(0.01, 0.25),
    'model__max_depth': randint(3, 10),
    'model__subsample': uniform(0.7, 0.3),
    'model__colsample_bytree': uniform(0.7, 0.3),
    'model__min_child_weight': randint(1, 7),
    'model__gamma': uniform(0, 5),
}

random_search = RandomizedSearchCV(
    xgb_pipeline,
    param_distributions=xgb_random_params,
    n_iter=50,
    cv=2,
    n_jobs=-1,
    verbose=2,
    scoring="f1"
)

Decidí dejar corriendo el Randomized Search durante bastante tiempo, por lo que dejé un print del output luego para poder persistir los resultados:

In [ ]:
random_search.fit(X_train,y_train)

Fitting 2 folds for each of 50 candidates, totalling 100 fits


/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


RandomizedSearchCV(cv=2,
                   estimator=Pipeline(steps=[('prep',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('keyword_ohe',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['keyword']),
                                                                              ('tweetlen_ohe',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['tweet_length']),
                                                                              ('location_tme',
                                                                               TargetEncoder(),
                                                                               ['standard_location']),
                                                                              ('location_fe',
                                                                               FunctionTransformer(featur...
                                        'model__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c257381f1a0>,
                                        'model__min_child_weight': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c257382f470>,
                                        'model__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c25875cd7f0>,
                                        'model__subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c257382f5f0>},
                   verbose=2)

In [ ]:
results = pd.DataFrame(random_search.cv_results_)

# Ordenar por F1 descendente
results_sorted = results.sort_values(by="mean_test_score", ascending=False)

# Top 10
top10 = results_sorted.head(10)
print(top10[["mean_test_score", "std_test_score"] +
          [c for c in results.columns if c.startswith("param_")]])


    mean_test_score  std_test_score  param_model__colsample_bytree  \
10         0.768637        0.004105                       0.896109   
20         0.766995        0.000821                       0.848379   
30         0.764368        0.005090                       0.816616   
3          0.764039        0.003120                       0.726387   
28         0.762233        0.004269                       0.848165   
17         0.762069        0.004105                       0.817315   
44         0.758949        0.002299                       0.880016   
21         0.757635        0.000657                       0.700738   
40         0.756486        0.017241                       0.756093   
37         0.754844        0.007718                       0.714220   

    param_model__gamma  param_model__learning_rate  param_model__max_depth  \
10            0.083262                    0.117666                       5   
20            0.085741                    0.165688                       

In [ ]:
best_model = random_search.best_estimator_
best_model

Pipeline(steps=[('prep',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('keyword_ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['keyword']),
                                                 ('tweetlen_ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['tweet_length']),
                                                 ('location_tme',
                                                  TargetEncoder(),
                                                  ['standard_location']),
                                                 ('location_fe',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<fun...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               learning_rate=np.float64(0.11766624194414536),
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None, min_child_weight=2,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=389,
                               n_jobs=None, num_parallel_tree=None, ...))])

## Top10 modelos con Random Search

Con un prompt de GPT pudimos pasar los prints a un array de diccionarios para recuperarlos rápidamente. Luego re-instanciamos los modelos, fiteamos con X_train + ColumnTransformer y hacemos los predicts para ver el score f1 de cada uno.

In [ ]:
# Lista de los 10 mejores hiperparámetros (copiando del print)
top10_params = [
    {'colsample_bytree': 0.896109, 'gamma': 0.083262, 'learning_rate': 0.117666, 'max_depth': 5,
     'min_child_weight': 2, 'n_estimators': 389, 'subsample': 0.764564},
    {'colsample_bytree': 0.848379, 'gamma': 0.085741, 'learning_rate': 0.165688, 'max_depth': 9,
     'min_child_weight': 3, 'n_estimators': 300, 'subsample': 0.834814},
    {'colsample_bytree': 0.816616, 'gamma': 1.179899, 'learning_rate': 0.231919, 'max_depth': 3,
     'min_child_weight': 5, 'n_estimators': 281, 'subsample': 0.887642},
    {'colsample_bytree': 0.726387, 'gamma': 0.135880, 'learning_rate': 0.241815, 'max_depth': 7,
     'min_child_weight': 4, 'n_estimators': 301, 'subsample': 0.712693},
    {'colsample_bytree': 0.848165, 'gamma': 0.915662, 'learning_rate': 0.030903, 'max_depth': 5,
     'min_child_weight': 1, 'n_estimators': 364, 'subsample': 0.701291},
    {'colsample_bytree': 0.817315, 'gamma': 3.823265, 'learning_rate': 0.104733, 'max_depth': 4,
     'min_child_weight': 6, 'n_estimators': 262, 'subsample': 0.779033},
    {'colsample_bytree': 0.880016, 'gamma': 1.495241, 'learning_rate': 0.076974, 'max_depth': 6,
     'min_child_weight': 4, 'n_estimators': 189, 'subsample': 0.813642},
    {'colsample_bytree': 0.700738, 'gamma': 1.776029, 'learning_rate': 0.220909, 'max_depth': 3,
     'min_child_weight': 6, 'n_estimators': 347, 'subsample': 0.750084},
    {'colsample_bytree': 0.756093, 'gamma': 3.174094, 'learning_rate': 0.078206, 'max_depth': 4,
     'min_child_weight': 3, 'n_estimators': 382, 'subsample': 0.877511},
    {'colsample_bytree': 0.714220, 'gamma': 1.950925, 'learning_rate': 0.051100, 'max_depth': 5,
     'min_child_weight': 3, 'n_estimators': 276, 'subsample': 0.765790}
]

In [ ]:
models = []

# Entrenar y evaluar cada modelo
for i, params in enumerate(top10_params, 1):
    model = XGBClassifier(eval_metric='logloss', random_state=123, **params)
    model.fit(X_train_transformed, y_train)
    models.append(model)

    y_train_pred = model.predict(X_train_transformed)
    y_validation_pred = model.predict(X_validation_transformed)

    f1_train = f1_score(y_train, y_train_pred)
    f1_val = f1_score(y_validation, y_validation_pred)

    print(f"Modelo {i}: F1 train = {f1_train:.4f}, F1 val = {f1_val:.4f}, Params = {params}")

Modelo 1: F1 train = 0.9994, F1 val = 0.7292, Params = {'colsample_bytree': 0.896109, 'gamma': 0.083262, 'learning_rate': 0.117666, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 389, 'subsample': 0.764564}
Modelo 2: F1 train = 0.9996, F1 val = 0.7133, Params = {'colsample_bytree': 0.848379, 'gamma': 0.085741, 'learning_rate': 0.165688, 'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 300, 'subsample': 0.834814}
Modelo 3: F1 train = 0.9962, F1 val = 0.7229, Params = {'colsample_bytree': 0.816616, 'gamma': 1.179899, 'learning_rate': 0.231919, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 281, 'subsample': 0.887642}
Modelo 4: F1 train = 0.9996, F1 val = 0.7246, Params = {'colsample_bytree': 0.726387, 'gamma': 0.13588, 'learning_rate': 0.241815, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 301, 'subsample': 0.712693}
Modelo 5: F1 train = 0.9875, F1 val = 0.7140, Params = {'colsample_bytree': 0.848165, 'gamma': 0.915662, 'learning_rate': 0.030903, 'max_dept

Viendo los primeros tres prints, ya podemos intuir que tenemos **overfitting**. El F1 en train está casi perfecto (0.99–1.0), pero en validation cae al rededor de 0.71–0.73. Esto indica que los modelos están aprendiendo demasiado los patrones de entrenamiento y no generalizan bien.

Ya que demoraron mucho en ajustarse los modelos, mejor no descartarlos y vamos a hacer un refinamiento más. Me voy a quedar con el modelo que menos overfitean (Modelo 6) y que a su vez tiene un score inicial razonable. Voy a hacer un GridSearch a su alrededor, bajando la cantidad de n_estimators que es lo que puede estar generando que el modelo aprenda ruido y overfitee. Vamos a ver si podemos acercar el score F1 en validation al 0.8. En caso de que no sea posible tendré que volver a entrenarlo.

## GridSearch para refinar los hiperparámetros.

In [ ]:
models[5]

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.817315, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, feature_weights=None,
              gamma=3.823265, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.104733,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=262, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
base_model = models[5]
params = base_model.get_params()

Ahora uso los hiperparámetros del random forest pero disminuyendo el n_estimators y voy a redondearlos por comodidad.
Otro approach si esto no funciona sería utilizar el Modelo 1 u otro de los que mejor le fue en validation y bajarle los n_estimators o max_depth para que disminuya la complejidad/el overfitting

### Iteración 1 de GridSearch

In [ ]:
xgb_base = XGBClassifier(
    gamma=3.8,
    learning_rate=0.1,
    min_child_weight=6,
    random_state=123,
    eval_metric='logloss'
)

grid_params = {
    'n_estimators': [150, 200],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [3, 4]
}

In [ ]:
grid_search_xgb = GridSearchCV(
    xgb_base,
    param_grid=grid_params,
    cv=2,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

In [ ]:
grid_search_xgb.fit(X_train_transformed, y_train)

Fitting 2 folds for each of 16 candidates, totalling 32 fits


GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=3.8,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=6,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8], 'max_depth': [3, 4],
                         'n_estimators': [150, 200], 'subsample': [0.7, 0.8]},
             scoring='f1', verbose=2)

Predicciones finales y metricas F1

In [ ]:
grid_search_xgb_model = grid_search_xgb.best_estimator_

In [ ]:
y_train_pred_xgb = grid_search_xgb_model.predict(X_train_transformed)
y_val_pred_xgb = grid_search_xgb_model.predict(X_validation_transformed)

In [ ]:
f1_train = f1_score(y_train, y_train_pred_xgb, average='weighted')
f1_val = f1_score(y_validation, y_val_pred_xgb, average='weighted')
gap = f1_train - f1_val

print(f"Modelo 6: F1 train = {f1_train:.4f}, F1 val = {f1_val:.4f}, gap = {gap:.4f}")

Modelo 6: F1 train = 0.9422, F1 val = 0.7646, gap = 0.1776


Podemos ver que peor score en train nos dio un mucho mejor score en validation. Tenemos que seguir ajustando y ya estamos cerca del 0.8

In [ ]:
best_params = grid_search_xgb.best_params_
best_params

{'colsample_bytree': 0.8,
 'max_depth': 3,
 'n_estimators': 200,
 'subsample': 0.8}

Podemos ver que relajando los n_estimators el score en validation subió bastante, aunque en train baja. Esto implica que GridSearch siempre va a estar midiendo en training y nos da un peor score en validation por eso. Para solucionar esto voy a aumentar las cross validations (parámetro cv), aunque esto aumentará considerablemente el tiempo de cómputo.

### Iteración 2

In [ ]:
xgb_base_2 = XGBClassifier(
    gamma=3.8,
    learning_rate=0.1,
    min_child_weight=6,
    random_state=123,
    eval_metric='logloss'
)

grid_params_2 = {
    'n_estimators': [100, 150, 200],
    'subsample': [0.75, 0.8],
    'colsample_bytree': [0.75, 0.8],
    'max_depth': [2, 3, 4]
}

In [ ]:
grid_search_xgb_2 = GridSearchCV(
    xgb_base_2,
    param_grid=grid_params_2,
    cv=4,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

In [ ]:
grid_search_xgb_2.fit(X_train_transformed, y_train)

Fitting 4 folds for each of 36 candidates, totalling 144 fits


GridSearchCV(cv=4,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=3.8,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=6,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.75, 0.8],
                         'max_depth': [2, 3, 4],
                         'n_estimators': [100, 150, 200],
                         'subsample': [0.75, 0.8]},
             scoring='f1', verbose=2)

Predicciones finales y metricas F1

In [ ]:
grid_search_xgb_model_2 = grid_search_xgb_2.best_estimator_

In [ ]:
y_train_pred_xgb_2 = grid_search_xgb_model_2.predict(X_train_transformed)
y_val_pred_xgb_2 = grid_search_xgb_model_2.predict(X_validation_transformed)

In [ ]:
f1_train = f1_score(y_train, y_train_pred_xgb_2, average='weighted')
f1_val = f1_score(y_validation, y_val_pred_xgb_2, average='weighted')
gap = f1_train - f1_val

print(f"Modelo 6: F1 train = {f1_train:.4f}, F1 val = {f1_val:.4f}, gap = {gap:.4f}")

Modelo 6: F1 train = 0.9392, F1 val = 0.7642, gap = 0.1750


Esto ya overfittea un poco menos pero también nos dio un puntaje un poco menor. Pero bajo el gap, lo que es bueno.

In [ ]:
grid_search_xgb_2.best_params_

{'colsample_bytree': 0.75,
 'max_depth': 3,
 'n_estimators': 200,
 'subsample': 0.75}

### Iteraciones recurrentes

Ahora voy a aplicar varias iteraciones más tratando de lograr un gap menor y un score mejor de forma recursiva. Asique no voy a persistir las celdas en el collab para evitar hacerlo demasiado largo. Voy dejando abajo algunos resultados que fui obteniendo en mis entrenos:

1)

Hiperparámetros:
```
{'colsample_bytree': 0.75,
 'gamma': 3.8,
 'max_depth': 3,
 'n_estimators': 250,
 'subsample': 0.75}
```
 Scores: F1 train = 0.9538, F1 val = 0.7696, gap = 0.1842

2. Luego de +1h de entreno con estos hiperparámetros:

In [ ]:
xgb_rec = XGBClassifier(
    learning_rate=0.1,
    random_state=123,
    eval_metric='logloss'
)

grid_params_rec = {
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_weight': [6],
    'n_estimators': [150, 200, 250],
    'subsample': [0.75],
    'colsample_bytree': [0.75],
    'max_depth': [3],
    'gamma': [3.8],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [1, 1.5, 2]

}

Gridsearch nos dio el mismo resultado:

Hiperparámetros:
```
{'colsample_bytree': 0.75,
 'gamma': 3.8,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': 6,
 'n_estimators': 250,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'subsample': 0.75}
```
 Scores: F1 train = 0.9538, F1 val = 0.7696, gap = 0.1842

3.
Asique decidí aumentar en 1 el cross-validation para evitar aún más el overfitting, disminuir a 200 el máximo de n_estimators y darle combinaciones con un menor subsample y colsample_bytree lo que puede llegar a bajar el overfitting. Esto me da 80 fits que son menos de los 334 que calculé para el paso anterior y va a demorar menos tiempo.

Los resultados fueron:

Hiperparámetros:
```
{'colsample_bytree': 0.6,
 'gamma': 3.8,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': 6,
 'n_estimators': 200,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'subsample': 0.6}
```
Score: F1 train = 0.9347, F1 val = 0.7635, gap = 0.1713

4. Decidí para la siguiente iteración:
- Agregar el colsample_bytree de 0.55, pero no más porque puede caer mucho el score.
- Bajar el learning_rate: vuelvo a agregar opciones 0.05 y 0.1
- Aumentar los n_estimators: Pruebo con 200 y 300.
- Agregar como opcion que max_depth sea 2, lo que puede llegar a sobresimplificar el modelo pero capaz ayude con el overfitting.
- Agregar un min_child_weight de 8 a las opciones.

El resultado fue:

```
 {'colsample_bytree': 0.6,
 'gamma': 3.8,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': 6,
 'n_estimators': 300,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'subsample': 0.6}
```
Score: F1 train = 0.9639, F1 val = 0.7656, gap = 0.1983


5. Esto subió el score pero también el overfitting. Asique decidí correrlo de nuevo, tomando los hiperparámetros anteriores pero forzando learning_rate=0.05, min_child_weight=[6,8] y max_depth=[2,3] y ver qué pasa.

El resultado fue:
```
{'colsample_bytree': 0.6,
 'gamma': 3.8,
 'learning_rate': 0.05,
 'max_depth': 3,
 'min_child_weight': 8,
 'n_estimators': 300,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'subsample': 0.6}
```
Score: F1 train = 0.9201, F1 val = 0.7569, gap = 0.1633


6. Lo que disminuyó el overfitting pero también el score. Cómo puedo seguir?

### Codigo e Hiperparámetros recursivos:

In [ ]:
xgb_rec = XGBClassifier(
    learning_rate=0.1,
    random_state=123,
    eval_metric='logloss'
)

grid_params_rec = {
    'learning_rate': [0.05],
    'min_child_weight': [6, 8],
    'n_estimators': [300],
    'subsample': [0.6],
    'colsample_bytree': [0.6],
    'max_depth': [2, 3],
    'gamma': [3.8],
    'reg_alpha': [0],
    'reg_lambda': [1],
    'objective': ['binary:logistic']
}

In [ ]:
grid_search_xgb_rec = GridSearchCV(
    xgb_rec,
    param_grid=grid_params_rec,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

Entreno recursivo:

In [ ]:
grid_search_xgb_rec.fit(X_train_transformed, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraint...
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6], 'gamma': [3.8],
                         'learning_rate': [0.05], 'max_depth': [2, 3],
                         'min_child_weight': [6, 8], 'n_estimators': [300],
                         'objective': ['binary:logistic'], 'reg_alpha': [0],
                         'reg_lambda': [1], 'subsample': [0.6]},
             scoring='f1', verbose=2)

Predicciones y metricas F1:

In [ ]:
grid_search_xgb_model_rec = grid_search_xgb_rec.best_estimator_

In [ ]:
y_train_pred_xgb_rec = grid_search_xgb_model_rec.predict(X_train_transformed)
y_val_pred_xgb_rec = grid_search_xgb_model_rec.predict(X_validation_transformed)

In [ ]:
f1_train = f1_score(y_train, y_train_pred_xgb_rec, average='weighted')
f1_val = f1_score(y_validation, y_val_pred_xgb_rec, average='weighted')
gap = f1_train - f1_val

print(f"Score: F1 train = {f1_train:.4f}, F1 val = {f1_val:.4f}, gap = {gap:.4f}")

Score: F1 train = 0.9201, F1 val = 0.7569, gap = 0.1633


Esto ya overfittea un poco menos pero también nos dio un puntaje un poco menor. Pero bajo el gap, lo que es bueno.

In [ ]:
grid_search_xgb_rec.best_params_

{'colsample_bytree': 0.6,
 'gamma': 3.8,
 'learning_rate': 0.05,
 'max_depth': 3,
 'min_child_weight': 8,
 'n_estimators': 300,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'subsample': 0.6}

## Checkpoint

En este momento, tomé la decisión de ponerme a trabajar con el siguiente modelo en el [colab de Random Forest](https://colab.research.google.com/drive/1eG9ansDIJDkN28k-wDn9jcKrvFtBsqh7?usp=sharing). Esto llevo a que realice la extracción de nuevas features que creo que aumentaron el resultado de la predicción considerablemente. Habría que ver ese colab primero antes de continuar con este.


### Mejor Modelo

Hasta ahora, el XGBoost que mejor resultados nos dio fue:

Hiperparámetros:
```
{'colsample_bytree': 0.75,
 'gamma': 3.8,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': 6,
 'n_estimators': 250,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'subsample': 0.75}
```
 Scores: F1 train = 0.9538, F1 val = 0.7696, gap = 0.1842

Un puntaje de casi 0.77 en validación está bastante bien. Pero considero que las features que agregué en RandomForest son bastante útiles y le serviran también a XGBoost para incrementar el puntaje.

Antes de continuar vamos a dejar instanciado el mejor XGBoost que tenemos hasta ahora:

In [ ]:
xgb_best = XGBClassifier(
  colsample_bytree= 0.75,
  gamma= 3.8,
  learning_rate= 0.1,
  max_depth= 3,
  min_child_weight= 6,
  n_estimators= 250,
  reg_alpha= 0,
  reg_lambda= 1,
  subsample= 0.75,
  random_state=123,
  eval_metric='logloss',
  early_stopping_rounds=30
)

In [ ]:
xgb_best.fit(X_train_transformed, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.75, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=3.8,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=250, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
y_pred_best = xgb_best.predict(X_validation_transformed)

In [ ]:
f1_val_best = f1_score(y_validation, y_pred_best, average='weighted')
print(f"Best XGBoost Score: F1 validation = {f1_val_best:.4f}")

Best XGBoost Score: F1 validation = 0.7696


In [ ]:
xgb_best.fit(
    X_train_transformed, y_train,
    eval_set=[(X_validation_transformed, y_validation)],
    verbose=True
)

[0]	validation_0-logloss:0.67634
[1]	validation_0-logloss:0.67046
[2]	validation_0-logloss:0.65946
[3]	validation_0-logloss:0.65541
[4]	validation_0-logloss:0.64801
[5]	validation_0-logloss:0.64449
[6]	validation_0-logloss:0.64290
[7]	validation_0-logloss:0.64096
[8]	validation_0-logloss:0.63337
[9]	validation_0-logloss:0.63063
[10]	validation_0-logloss:0.62764
[11]	validation_0-logloss:0.62102
[12]	validation_0-logloss:0.61986
[13]	validation_0-logloss:0.61875
[14]	validation_0-logloss:0.61656
[15]	validation_0-logloss:0.61472
[16]	validation_0-logloss:0.60971
[17]	validation_0-logloss:0.60815
[18]	validation_0-logloss:0.60515
[19]	validation_0-logloss:0.60430
[20]	validation_0-logloss:0.60156
[21]	validation_0-logloss:0.59911
[22]	validation_0-logloss:0.59547
[23]	validation_0-logloss:0.59150
[24]	validation_0-logloss:0.58859
[25]	validation_0-logloss:0.58823
[26]	validation_0-logloss:0.58488
[27]	validation_0-logloss:0.58249
[28]	validation_0-logloss:0.58076
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.75, device=None, early_stopping_rounds=30,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=3.8,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=250, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
f1_val_best = f1_score(y_validation, y_pred_best, average='weighted')
print(f"Best XGBoost Score Post Early Stopping: F1 validation = {f1_val_best:.4f}")

Best XGBoost Score Post Early Stopping: F1 validation = 0.7696


Hacer un early stopping tampoco mejora.

## Boosteando XGBoost

### Agregando features de keyword

Como este puntaje es peor que el mejor obtenido en RandomForest, por ahora no voy a predecir test. Ahora vamos a tratar de mejorar XGBoost con los nuevos features:

Latitudes y longitudes:

In [ ]:
X_validation_latlon = pd.read_csv("../data/processed/X_validation_latlon.csv", index_col=0)
X_train_latlon = pd.read_csv("../data/processed/X_train_latlon.csv", index_col=0)
X_test_latlon = pd.read_csv("../data/processed/X_test_latlon.csv", index_col=0)

In [ ]:
X_train = X_train.merge(X_train_latlon, how="left", left_index=True, right_index=True)

In [ ]:
X_validation = X_validation.merge(X_validation_latlon, how="left", left_index=True, right_index=True)

In [ ]:
X_test = X_test.merge(X_test_latlon, how="left", left_index=True, right_index=True)

Keyword features:

In [ ]:
X_train_keyword_feats = pd.read_csv("../data/processed/X_train_keyword_features.csv", index_col=0)

In [ ]:
X_validation_keyword_feats = pd.read_csv("../data/processed/X_validation_keyword_features.csv", index_col=0)

In [ ]:
X_test_keyword_feats = pd.read_csv("../data/processed/X_test_keyword_features.csv", index_col=0)

In [ ]:
X_train = X_train.merge(X_train_keyword_feats, how="left", left_index=True, right_index=True)

In [ ]:
X_validation = X_validation.merge(X_validation_keyword_feats, how="left", left_index=True, right_index=True)

In [ ]:
X_test = X_test.merge(X_test_keyword_feats, how="left", left_index=True, right_index=True)

In [ ]:
X_train = X_train.drop(columns=['keyword'])
X_validation = X_validation.drop(columns=['keyword'])
X_test = X_test.drop(columns=['keyword'])

Como XGBoost está basado en arboles, tampoco tenía mucho sentido escalar las features:

In [ ]:
categorical_keyword = ["keyword"]
categorical_tweetlen = ["tweet_length"]
categorical_location = ["standard_location"]
numerical_cols = cols_numericas = X_train.select_dtypes(include=['number']).columns

In [ ]:
preprocessor_2 = ColumnTransformer([
    ("tweetlen_ohe", OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_tweetlen),
    ("location_tme", TargetEncoder(handle_unknown='value', handle_missing='value'), categorical_location),
    ("location_fe", freq_encoder, categorical_location),
    ("numerical_passthrough", "passthrough", numerical_cols),
], remainder="passthrough")

In [ ]:
X_train_transformed_2 = preprocessor_2.fit_transform(X_train, y_train)
X_validation_transformed_2 = preprocessor_2.transform(X_validation)

In [ ]:
X_test_transformed_2 = preprocessor_2.transform(X_test)

ValueError: columns are missing: {'text'}

### Entrenando con nuevas features

Vamos a probar nuestro mejor modelo con estas nuevas features. Para eso, hago un random search:

In [ ]:
xgb_best_2 = XGBClassifier(
  colsample_bytree= 0.75,
  gamma= 3.8,
  learning_rate= 0.1,
  max_depth= 3,
  min_child_weight= 6,
  n_estimators= 1000,
  reg_alpha= 0,
  reg_lambda= 1,
  subsample= 0.75,
  random_state=123,
  eval_metric='logloss',
  tree_method="hist",
)

In [ ]:
param_distributions = {
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "max_depth": [3, 4, 5, 6],
    "min_child_weight": [1, 2, 4, 6, 8],
    "gamma": [0, 0.5, 1, 2, 3, 5],
    "subsample": [0.6, 0.7, 0.75, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.7, 0.75, 0.8, 1.0],
    "reg_alpha": [0, 0.1, 0.5, 1],
    "reg_lambda": [0.5, 1, 2]
}

In [ ]:
random_search = RandomizedSearchCV(
    estimator=xgb_best_2,
    param_distributions=param_distributions,
    n_iter=20,
    scoring="f1_weighted",
    cv=4,
    verbose=2,
    n_jobs=-1,
    random_state=123
)

In [ ]:
random_search.fit(
    X_train_transformed_2,
    y_train,
)

Fitting 4 folds for each of 20 candidates, totalling 80 fits


RandomizedSearchCV(cv=4,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=0.75, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=3.8,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_const...
                                           num_parallel_tree=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.75,
                                                             0.8, 1.0],
                                        'gamma': [0, 0.5, 1, 2, 3, 5],
                                        'learning_rate': [0.01, 0.03, 0.05,
                                                          0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'min_child_weight': [1, 2, 4, 6, 8],
                                        'reg_alpha': [0, 0.1, 0.5, 1],
                                        'reg_lambda': [0.5, 1, 2],
                                        'subsample': [0.6, 0.7, 0.75, 0.8,
                                                      1.0]},
                   random_state=123, scoring='f1_weighted', verbose=2)

In [ ]:
print("Mejores hyperparams:", random_search.best_params_)
print()
best_model_2 = random_search.best_estimator_
y_pred_rs = best_model_2.predict(X_validation_transformed_2)

f1_val_rs = f1_score(y_validation, y_pred_rs, average="weighted")
print(f"RandomSearch XGB Score: F1 validation = {f1_val_rs:.4f}")


Mejores hyperparams: {'subsample': 0.8, 'reg_lambda': 0.5, 'reg_alpha': 1, 'min_child_weight': 6, 'max_depth': 3, 'learning_rate': 0.03, 'gamma': 0.5, 'colsample_bytree': 1.0}

RandomSearch XGB Score: F1 validation = 0.8020


Copy del print de arriba por las dudas:
```
Mejores hyperparams: {'subsample': 0.8, 'reg_lambda': 0.5, 'reg_alpha': 1, 'min_child_weight': 6, 'max_depth': 3, 'learning_rate': 0.03, 'gamma': 0.5, 'colsample_bytree': 1.0}

RandomSearch XGB Score: F1 validation = 0.8020
```

Excelente! Con estas nuevas features y un XGBoost entrenado con el descenso por gradiente, obtuvimos al fin un puntaje de 0.8 en validación!

## Prediciendo Test

En el entreno a última hora, el modelo de XGBoost boosteado superó el 0.7923 en validation que tenía mi RandomForest! Siendo el mejor score de 0.8020 en validación, vuelvo a hacer un submit en kaggle.

Por falta de tiempo, no llegue a hacer submit en Kaggle de este modelo antes de que cierre la fecha de entrega del tp.

De aca para abajo quedo WIP que seria predecir en test y hacer submit a Kaggle.